In [6]:
import pandas as pd
import json
import jsonlines
from datasets import Dataset
import os
import platform

In [9]:
os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'

DATA_PATH = PRE_PATH + "data/ollama/"
csv_file_path = os.path.join(DATA_PATH, f"indata_kor.csv")
df = pd.read_csv(csv_file_path, encoding="utf-8")

In [10]:
df["input"]=""
df.columns = ["instruction","output","input"]
df.head(2)

,instruction,output,input
0,유튜브 채널 hkcode에서는 무엇을 가르치나요?,"초보자 대상으로 빅데이터, 인공지능과 관련된 컨텐츠를 가르치고 있습니다.",
1,유튜브 채널 hkcode는 누가 운영하나요?,한국폴리텍대학 스마트금융과 김효관 교수가 운영합니다.,


In [11]:
dataset = Dataset.from_pandas(df)

In [13]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
# datasethk = load_dataset(dataset, split="train")
datasethk = dataset

def create_text_column(example):
    # 'text' 컬럼 생성
    text = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    example["text"] = text
    return example

# 'text' 컬럼 생성
datasethk = datasethk.map(create_text_column)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [14]:
datasethk[10]

{'instruction': '스마트금융과를 수료하면 어떤 포트폴리오가 나오나요?',
 'output': '스마트금융과는 찍어내기식의 포트폴리오가 아니라 매년 업체에서 요구하는 기술 및 주제에 대해서 포트폴리오가 나옵니다. 2024년2월에는 AWS 사용량 예측을 진행하기도 했습니다.',
 'input': '',
 'text': '### Instruction:\n스마트금융과를 수료하면 어떤 포트폴리오가 나오나요?\n\n### Response:\n스마트금융과는 찍어내기식의 포트폴리오가 아니라 매년 업체에서 요구하는 기술 및 주제에 대해서 포트폴리오가 나옵니다. 2024년2월에는 AWS 사용량 예측을 진행하기도 했습니다.'}

In [18]:
import huggingface_hub
huggingface_hub.login()

In [19]:
if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

# BitsAndBytesConfig 객체활용 양자화 설정
# 모델을 4비트 양자화하여 로드
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

In [21]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
    # device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

#  https://llama3-1.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoibWYxOXZrN3g4eHpjamxoZ291c3loNDN4IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvbGxhbWEzLTEubGxhbWFtZXRhLm5ldFwvKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNTc1OTU1Nn19fV19&Signature=mhA%7EJHyTUGw4NiDakIF4TEaFdlzJ%7Ef1ww-Xhx5bqX1eD3kpQwhjmwYFX48g%7E6neJgbV904NoOXygTWAHHs8Cf2x17VgCIyQOPKjfuqb7QeX0qTM1GCqrfcIxK19D3xZwRX6DfzWj0WF94eMFrSC9eXLvAgTh64oPQFHLbTPqr7mOk1mAkfIOwKqilmIFbWMBnQFicEaOtHZ9TdyfJqhErxfLqE6gYm%7Ew82E-WOqdXp%7EQrSnaTolSqPP6R09k9Jn%7E%7ED7GMsirIUr0rHxcM06c6me6Fv9vB0uF4CYHg4yGLfa4m6034-a89k2zF86aubmKO1n-LhNVdFkYfRC4-NbhdA__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=933353511949348

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
403 Client Error. (Request ID: Root=1-66dbba3c-191ed9a63132bd967ecd71a7;379eeac5-c92f-4f05-9ba9-62de16e90190)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Your request to access model meta-llama/Meta-Llama-3-8B-Instruct is awaiting a review from the repo authors.

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# Must add EOS_TOKEN at response last line
tokenizer.pad_token = tokenizer.eos_token
# ★수정 포인트!!! 기존 # tokenizer.padding_side = "right"
EOS_TOKEN = tokenizer.eos_token
def prompt_eos(sample):
    sample['text'] = sample['text']+EOS_TOKEN
    return sample
datasethk = datasethk.map(prompt_eos)

In [ ]:
datasethk[1]